In [5]:
# 原理的なコード

import time
from functools import wraps

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"{func.__name__}の実行時間: {execution_time:.6f}秒")
        return result
    return wrapper

# 使用例
@measure_time
def example_function(n):
    total = 0
    for i in range(n):
        total += i
    return total

# テスト
result = example_function(1000000)
print(f"結果: {result}")

example_functionの実行時間: 0.030009秒
結果: 499999500000


In [1]:
# 複数メソッドの計測

import time
from functools import wraps
from collections import defaultdict

class PerformanceTracker:
    def __init__(self):
        self.execution_times = defaultdict(list)

    def measure_time(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time
            self.execution_times[func.__name__].append(execution_time)
            return result
        return wrapper

    def print_stats(self):
        print("\n性能統計:")
        for func_name, times in self.execution_times.items():
            avg_time = sum(times) / len(times)
            min_time = min(times)
            max_time = max(times)
            print(f"{func_name}:")
            print(f"  平均実行時間: {avg_time:.6f}秒")
            print(f"  最小実行時間: {min_time:.6f}秒")
            print(f"  最大実行時間: {max_time:.6f}秒")
            print(f"  呼び出し回数: {len(times)}")

# 使用例
tracker = PerformanceTracker()

@tracker.measure_time
def function1(n):
    return sum(range(n))

@tracker.measure_time
def function2(n):
    return [i**2 for i in range(n)]

# テスト
for _ in range(100):
    function1(10000)
    function2(1000)

tracker.print_stats()


性能統計:
function1:
  平均実行時間: 0.000088秒
  最小実行時間: 0.000086秒
  最大実行時間: 0.000112秒
  呼び出し回数: 100
function2:
  平均実行時間: 0.000184秒
  最小実行時間: 0.000179秒
  最大実行時間: 0.000378秒
  呼び出し回数: 100


In [ ]:
# 毎秒計測

import time
from functools import wraps
from collections import defaultdict
import threading

class PerformanceTracker:
    def __init__(self, update_interval=1):
        self.execution_times = defaultdict(list)
        self.update_interval = update_interval
        self.stop_flag = threading.Event()
        self.lock = threading.Lock()

    def measure_time(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time
            with self.lock:
                self.execution_times[func.__name__].append(execution_time)
            return result
        return wrapper

    def print_stats(self):
        with self.lock:
            print("\n現在の性能統計:")
            for func_name, times in self.execution_times.items():
                if times:
                    avg_time = sum(times) / len(times)
                    min_time = min(times)
                    max_time = max(times)
                    print(f"{func_name}:")
                    print(f"  平均実行時間: {avg_time:.6f}秒")
                    print(f"  最小実行時間: {min_time:.6f}秒")
                    print(f"  最大実行時間: {max_time:.6f}秒")
                    print(f"  呼び出し回数: {len(times)}")
            print("\n")

    def start_real_time_tracking(self):
        def update_stats():
            while not self.stop_flag.is_set():
                self.print_stats()
                time.sleep(self.update_interval)

        self.tracking_thread = threading.Thread(target=update_stats)
        self.tracking_thread.start()

    def stop_real_time_tracking(self):
        self.stop_flag.set()
        if hasattr(self, 'tracking_thread'):
            self.tracking_thread.join()

# 使用例
tracker = PerformanceTracker(update_interval=1)  # 1秒ごとに更新

@tracker.measure_time
def function1(n):
    time.sleep(0.1)  # シミュレーションのための遅延
    return sum(range(n))

@tracker.measure_time
def function2(n):
    time.sleep(0.2)  # シミュレーションのための遅延
    return [i**2 for i in range(n)]

# テスト
tracker.start_real_time_tracking()

try:
    for _ in range(10):
        function1(10000)
        function2(1000)
        time.sleep(0.5)  # 処理間隔を設定
finally:
    tracker.stop_real_time_tracking()

print("テスト終了")

In [ ]:
# 複数ファイルに対応したバージョン
# シングルトンパターンになっている
# performance_tracker.py

import time
from functools import wraps
from collections import defaultdict
import threading

class PerformanceTracker:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(PerformanceTracker, cls).__new__(cls)
            cls._instance.execution_times = defaultdict(list)
            cls._instance.update_interval = 1
            cls._instance.stop_flag = threading.Event()
            cls._instance.lock = threading.Lock()
        return cls._instance

    def measure_time(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time
            with self.lock:
                self.execution_times[func.__name__].append(execution_time)
            return result
        return wrapper

    def print_stats(self):
        with self.lock:
            print("\n現在の性能統計:")
            for func_name, times in self.execution_times.items():
                if times:
                    avg_time = sum(times) / len(times)
                    min_time = min(times)
                    max_time = max(times)
                    print(f"{func_name}:")
                    print(f"  平均実行時間: {avg_time:.6f}秒")
                    print(f"  最小実行時間: {min_time:.6f}秒")
                    print(f"  最大実行時間: {max_time:.6f}秒")
                    print(f"  呼び出し回数: {len(times)}")
            print("\n")

    def start_real_time_tracking(self, interval=1):
        self.update_interval = interval
        def update_stats():
            while not self.stop_flag.is_set():
                self.print_stats()
                time.sleep(self.update_interval)

        self.tracking_thread = threading.Thread(target=update_stats)
        self.tracking_thread.start()

    def stop_real_time_tracking(self):
        self.stop_flag.set()
        if hasattr(self, 'tracking_thread'):
            self.tracking_thread.join()

# class1.py
from performance_tracker import PerformanceTracker

class Class1:
    def __init__(self):
        self.tracker = PerformanceTracker()

    @PerformanceTracker().measure_time
    def method1(self, n):
        time.sleep(0.1)  # シミュレーションのための遅延
        return sum(range(n))

# class2.py
from performance_tracker import PerformanceTracker

class Class2:
    def __init__(self):
        self.tracker = PerformanceTracker()

    @PerformanceTracker().measure_time
    def method2(self, n):
        time.sleep(0.2)  # シミュレーションのための遅延
        return [i**2 for i in range(n)]

# main.py
from performance_tracker import PerformanceTracker
from class1 import Class1
from class2 import Class2
import time

def main():
    tracker = PerformanceTracker()
    tracker.start_real_time_tracking(interval=1)

    obj1 = Class1()
    obj2 = Class2()

    try:
        for _ in range(10):
            obj1.method1(10000)
            obj2.method2(1000)
            time.sleep(0.5)  # 処理間隔を設定
    finally:
        tracker.stop_real_time_tracking()

    print("テスト終了")

if __name__ == "__main__":
    main()